In [5]:
!pip install osmnx
!pip install matplotlib
!pip install geopandas
!pip install folium

In [6]:
import folium
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from sklearn.cluster import DBSCAN
import numpy as np

In [8]:
def is_bike_friendly(edge):
    # Consider paths where bikes are explicitly allowed
    bike_friendly_tags = ['yes', 'designated', 'permissive', 'lane', 'shared', 'shared_lane']
    bike_path_types = ['cycleway', 'path', 'living_street', 'residential']

    # Check for bicycle tag or highway type
    is_bike_tag_friendly = edge.get('bicycle') in bike_friendly_tags
    is_highway_type_friendly = edge.get('highway') in bike_path_types

    # Check for specific infrastructure for bikes
    has_cycleway_infrastructure = ('cycleway' in edge) or ('cycleway:right' in edge) or ('cycleway:left' in edge)

    # Additional check for footways and pedestrian paths if explicitly marked as bike-friendly
    is_footway_bike_friendly = (edge.get('highway') in ['footway', 'pedestrian']) and edge.get('bicycle') == 'yes'

    return is_bike_tag_friendly or is_highway_type_friendly or has_cycleway_infrastructure or is_footway_bike_friendly

place_name = "Münster, Germany"
G = ox.graph_from_place(place_name, network_type='all')
df = pd.read_csv('filtered_data_munster.csv')

# Identificar paradas de ônibus
tags_bus = {'highway': 'bus_stop'}
bus_stops = ox.geometries_from_place(place_name, tags=tags_bus)

# Filtrar arestas amigáveis para bicicletas
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
bike_friendly_edges = edges[edges.apply(is_bike_friendly, axis=1)]

# Converter para um CRS projetado antes de criar buffers
crs_proj = 'EPSG:32632'  # UTM zone 32N, adequado para a Alemanha
bus_stops = bus_stops.to_crs(crs_proj)
bike_friendly_edges = bike_friendly_edges.to_crs(crs_proj)
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False).to_crs(crs_proj)

# Criar buffers de 20 metros
bus_stops_buffer = bus_stops.buffer(20)  # Para paradas de ônibus
bike_friendly_buffer = bike_friendly_edges.buffer(20)  # Para ciclovias

# Identificar nós próximos às paradas de ônibus e ciclovias
near_bus_stops = nodes[nodes.geometry.apply(lambda x: bus_stops_buffer.contains(x).any())]
near_bike_paths = nodes[nodes.geometry.apply(lambda x: bike_friendly_buffer.contains(x).any())]
near_both = near_bus_stops[near_bus_stops.index.isin(near_bike_paths.index)].to_crs('EPSG:4326')  # Convertendo de volta para o CRS geográfico

# Criar o mapa com Folium
map = folium.Map(location=[51.9625, 7.6256], zoom_start=13)

df_near_both = pd.DataFrame([(point.y, point.x) for point in near_both.geometry], columns=['Latitude', 'Longitude'])

# Aplicar DBSCAN para agrupar pontos próximos
# eps é o raio em graus, min_samples é o número mínimo de pontos para formar um cluster
dbscan = DBSCAN(eps=0.001, min_samples=1)  # Ajuste 'eps' conforme necessário
clusters = dbscan.fit_predict(df_near_both[['Latitude', 'Longitude']])

# Adicione as labels dos clusters ao DataFrame
df_near_both['Cluster'] = clusters

# Calcular o centro de cada cluster
cluster_centers = df_near_both.groupby('Cluster').mean()

# Agora, use 'cluster_centers' para seus marcadores no mapa
for _, row in cluster_centers.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=7,
        color='red',
        fill=True,
        tooltip='Cluster Center'
    ).add_to(map)

# Adicionar marcadores para dados adicionais do df
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,
        color='blue',
        fill=True
    ).add_to(map)

# Salvar e mostrar o mapa
map.save('analize_munster.html')
map

/var/folders/vz/b1smgzt13_dbs5c1rbj4m3dh0000gn/T/ipykernel_27254/1149520922.py:24: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  bus_stops = ox.geometries_from_place(place_name, tags=tags_bus)
